Now we'll look at a more interesting and clearly nonlinear example -- inferring an unknown conductivity rather than an unknown right-hand side.

In [ ]:
import firedrake
import firedrake_adjoint
mesh = firedrake.UnitSquareMesh(32, 32)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])
        
        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q_true += mode

Now to get the true solution of the PDE.

In [ ]:
from firedrake import exp, inner, grad, dx
u = firedrake.Function(V)
f = Constant(1.0)
J = (0.5 * exp(q_true) * inner(grad(u), grad(u)) - f * u) * dx
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)
u_true = u.copy(deepcopy=True)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u_true, axes=axes, shading='gouraud')
fig.colorbar(colors);